In [1]:
import skimage
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

After single cell segmentation on hosts cells, we can get the medium dsDNA expression level per cell in each FOV. this can serve as a baseline of the expression level, where the medium dsDNA (per cell) level should be roughly stable over FOVs. Based on this we perform normalization per FOV.

## create a run loop version

In [3]:
# create csv of single cell extracted

img_dir = '../../final_data/masked/micro_fine1ms/'
seg_dir = '../../final_data/masked_seg/micro_fine1ms/'
img_out = '../../final_data/normed/micro_fine1ms/'

#runs = ['2023-03-04T16-22-15-DSSC1-Run1']

#runs = ['2023-03-04T16-22-15-DSSC1-Run1', '2023-03-04T19-46-22-DSSC1-Run2',
#             '2023-03-04T23-36-59-DSSC1-Run3', '2023-03-05T15-51-42-DSST2-Run1',
#             '2023-03-05T18-22-42-DSST2-Run2', '2023-03-05T21-42-35-DSST2-Run3',
#             '2023-03-05T23-58-06-DSST2-Run4', '2023-03-06T11-23-28-DSST2-Run5',
#             '2023-03-06T13-40-46-DSST2-Run6', '2023-03-06T16-15-43-DSSC2-Run1',
#             '2023-03-07T18-32-49-DSSC2-Run5', '2023-03-07T16-08-39-DSSC2-Run4',
#             '2023-03-07T13-54-48-DSSC2-Run3', '2023-03-07T11-17-56-DSSC2-Run2',
#             '2023-03-09T15-53-36-DSSC3-Run4', '2023-03-09T13-34-52-DSSC3-Run3',
#             '2023-03-09T10-32-36-DSSC3-Run2', '2023-03-08T23-51-06-DSSC3-Run1',
#             '2023-03-08T21-26-09-DSST3-Run6', '2023-03-08T18-48-43-DSST3-Run5',
#             '2023-03-08T16-15-19-DSST3-Run4', '2023-03-08T14-03-50-DSST3-Run3',
#             '2023-03-08T11-39-28-DSST3-Run2', '2023-03-07T22-15-40-DSST3-Run1'] # specific run name

runs = ['2023-03-10T12-31-44-DSST4-Run1', '2023-03-10T14-47-38-DSST4-Run2', '2023-03-10T17-34-14-DSST4-Run3',
            '2023-03-18T14-36-10-DSST4-Run4', '2023-03-18T17-28-03-DSST4-Run5', '2023-03-18T23-11-36-DSST4-Run6',
            '2023-03-19T12-45-44-DSSC4-Run1', '2023-03-19T15-20-40-DSSC4-Run2', '2023-03-19T17-38-15-DSSC4-Run3',
            '2023-03-19T20-32-23-DSSC4-Run4', '2023-03-20T12-07-46-DSST1-Run1', '2023-03-20T14-32-22-DSST1-Run2',
            '2023-03-20T19-13-53-DSST1-Run3', '2023-03-20T21-44-51-DSST1-Run4', '2023-03-21T24-29-43-DSST1-Run5',
            '2023-03-21T10-32-42-DSST1-Run6', '2023-03-21T12-44-51-DSST1-Run7', '2023-03-28T11-45-15-DSSC1-Run4'
            ]

seg_subfld = '0.3mpp_0.065maxima_0.05interior' 

channels = ['dsDNA_host.tiff'] # channel for normalization calculation

channels_norm = ['dsDNA_host.tiff', 'TAMRA.tiff', 'Alx488.tiff', 'FITC.tiff', 'DIG.tiff',
                 'B220.tiff', 'CD3e.tiff', 'CD4.tiff', 'CD11b.tiff', 'CD11c.tiff', 'CD31.tiff',
                 'CD45.tiff', 'CD68.tiff', 'DCAMLK1.tiff', 'DIG.tiff', 'Ecad.tiff',
                'F480.tiff', 'FITC.tiff', 'IgA.tiff', 'Ki67.tiff', 'Ly6g.tiff', 'MUC2.tiff',
                'PNAD-1.tiff', 'Reg3beta.tiff', 'SMA.tiff', 'TAMRA.tiff', 'Tubulin.tiff', 'Vimentin.tiff',
                'chan_163.tiff','chan_140.tiff'] # last two empty channel used for agg rmv
                ## note 'dsDNA_feces.tiff' is not normalized
    
################### extract dsdna info to get ratio

for run in runs:
    container = []
    path_masked = os.path.join(img_dir, run)
    fovs = os.listdir(path_masked)
    
    for fov in fovs:
        print('calculating on ', fov)  
        if fov == '.DS_Store':
            continue
            
        mesmer_labels = skimage.io.imread( seg_dir + run + '/' + fov + '/' + seg_subfld + '/' + 'MESMER_mask.tiff')
        ###### first extract the basic infos
        csz = {}
        cx = {}
        cy = {}
        cd = {}
        regions = skimage.measure.regionprops(mesmer_labels)

        i = 1
        for cell in regions:
            csz[i] = cell.area
            x,y = cell.centroid
            cx[i] = x
            cy[i] = y
            i = i + 1
        
        df = pd.DataFrame([csz, cx,cy])
        df = df.transpose()
        df.columns = ['size', 'x', 'y']
        ###### basic info finished, start extract channel info

        for channel in channels:
            channel_img = skimage.io.imread( img_dir + run + '/' + fov + '/' + channel )

            for cell in np.unique(mesmer_labels):
                if cell == 0: # skip the empty bg cell
                    continue
                # extract signal
                sigxt = np.sum(channel_img[mesmer_labels == cell])
                cd[cell] = sigxt

            df[channel[:-5]] = cd
            df['fov'] = fov 
            container.append(df)
            
    ## now for each run you have a df ratio
    dfall = pd.concat(container)
    dfall['ds_ratio'] = dfall['dsDNA_host']/dfall['size']
    
    allmed = dfall.groupby('fov')['ds_ratio'].median()
    maxmed = np.max(dfall.groupby('fov')['ds_ratio'].median())             
    
    fovlist = (allmed/maxmed).index.tolist()
    fovratio = (allmed/maxmed).tolist()
    
    for i in range(len(fovlist)):
        fov = fovlist[i]
        fovr = fovratio[i]
        os.makedirs(img_out + run + '/' + fov, exist_ok=True)
        
        for channel in channels_norm:
            channel_img = skimage.io.imread( img_dir + run + '/' + fov + '/' + channel )
            channel_img_out = channel_img / fovr

            sv_dir = img_out + run + '/' + fov + '/' + channel
            skimage.io.imsave(sv_dir, channel_img_out)


calculating on  fov-2-scan-1
calculating on  .DS_Store
calculating on  fov-4-scan-1
calculating on  fov-3-scan-1
calculating on  fov-1-scan-1
calculating on  fov-5-scan-1
calculating on  fov-2-scan-1
calculating on  fov-4-scan-1
calculating on  fov-3-scan-1
calculating on  fov-1-scan-1
calculating on  fov-5-scan-1
calculating on  fov-2-scan-1
calculating on  fov-4-scan-1
calculating on  fov-3-scan-1
calculating on  fov-1-scan-1
calculating on  fov-5-scan-1
calculating on  fov-2-scan-1
calculating on  fov-4-scan-1
calculating on  fov-3-scan-1
calculating on  fov-1-scan-1
calculating on  fov-5-scan-1
calculating on  fov-2-scan-1
calculating on  fov-4-scan-1
calculating on  fov-3-scan-1
calculating on  fov-1-scan-1
calculating on  fov-5-scan-1
calculating on  fov-2-scan-1
calculating on  fov-4-scan-1
calculating on  fov-3-scan-1
calculating on  fov-1-scan-1
calculating on  fov-2-scan-1
calculating on  fov-4-scan-1
calculating on  fov-3-scan-1
calculating on  fov-1-scan-1
calculating on  f